In [1]:
!wget http://zerg1.fyre.ibm.com/FileShare/atlas_higgs.csv -q

## Spark Lazy Evaluation 


- spark.read : Will load the data schema/reference to Driver/Executors - No transofrm operations yet 
- df1.registerTempTable - Lazy Evaluation against original df. 
- df2 = spark.sql operations on df1 - Lazy Evaluation against df1
- df2.show() / df2.count() / df2.collect() -> Execute the DAG/Slow process

In [2]:
%%time
df1 = spark.read.csv('./atlas_higgs.csv', inferSchema = True, header = True)
# df1 = spark.sql("SELECT * FROM csv.`./atlas_higgs.csv`")

CPU times: user 8.41 ms, sys: 7.42 ms, total: 15.8 ms
Wall time: 21.3 s


**Lazy Eval Operations:**

In [3]:
%%time
df1.registerTempTable("in_mem_tmp_table")

CPU times: user 2.13 ms, sys: 233 µs, total: 2.36 ms
Wall time: 291 ms


In [4]:
%%time
df2 = spark.sql("""select EventId, DER_mass_MMC, DER_mass_vis 
                   from in_mem_tmp_table
                   where EventId > 100000 and EventId < 120000 and DER_mass_MMC > 1
                """)

CPU times: user 3.41 ms, sys: 1.96 ms, total: 5.37 ms
Wall time: 156 ms


**Collect Operations**

In [5]:
%%time
df2.count()

CPU times: user 4.2 ms, sys: 728 µs, total: 4.92 ms
Wall time: 7.07 s


16972

In [6]:
%%time
df1.count()

CPU times: user 2.71 ms, sys: 360 µs, total: 3.07 ms
Wall time: 2.14 s


250000

In [7]:
%%time
df2.show(1)

+-------+------------+------------+
|EventId|DER_mass_MMC|DER_mass_vis|
+-------+------------+------------+
| 100001|     160.937|     103.235|
+-------+------------+------------+
only showing top 1 row

CPU times: user 2.32 ms, sys: 1.56 ms, total: 3.88 ms
Wall time: 1.63 s


---

### Note, spark.sql will treat the df= as a lazy evaluation step for non-file datasources (JDBC/Hive/etc):

If the spark.read was spark.sql on an Existing Spark Datasource instead of csv, the "load" is optimized to Lazy Evaluation:

In [8]:
%%time
df_connected_hive = spark.sql("select * from atlas_higgs_demo")

CPU times: user 1 ms, sys: 1.13 ms, total: 2.14 ms
Wall time: 181 ms


In [9]:
%%time
df_connected_hive.count()

CPU times: user 6.44 ms, sys: 3.34 ms, total: 9.78 ms
Wall time: 1.93 s


250000